# File description
We annotated the videos using the open source annotation tool `cvat`. We had some problems regarding the export of all the annotations. The main problem was that we needed all annotation to be specifically labeled according to date, time and location of recording. This was not possible and we therefor had to find a creative solution. In `cvat` it's possible to take a backup of your projects and all the individual tasks contained within this project. This backup contain all the information we needed, but it was a pain to extract it, and included a lot of trial an error. But the main approach can be simplified into these steps:

    1.) Extract the video and its corresponding bounding boxes from the backup file
    2.) Match the location+date+video with a dataframe `df_video_info` containing precise time information on the videos
    3.) Extract the the precise time (hours+minuts) from `df_video_info` match
    4.) Break the videos into individual frames with their corresponding bounding box annotation 
    5.) Transform BBs into YOLO-format and save both images and BBs to disk
    6.) Make a dataframe with all available information 
<br>
<br>
The resulting dataframe looks something like this (Some columns and rows are missing)
<img src="../illustration_images/df_info_example.png" width="800" /> 


In [ ]:
import dutils as U
U.jupyter_ipython.adjust_screen_width(75)
from dutils.jupyter_ipython import show_image as show
import seaborn; seaborn.set_style("whitegrid")
from tqdm.notebook import tqdm

import random
import zipfile
from glob import glob
import pandas as pd
import json
import cv2
import os

# Setup - paths

In [ ]:
# Unpack zip file
backup_zip_path = "../cvat/project_bachelor_combined_backup_2022_04_13_21_38_08.zip"
backup_folder_path = backup_zip_path[:-4]
assert os.path.isfile(backup_zip_path) and (backup_zip_path[-4:] == ".zip"), "Backup file is invalid"
assert not os.path.exists(backup_folder_path), "Attempt to unpack into a folder that already exists"
with zipfile.ZipFile(backup_zip_path) as zip_file:
        zip_file.extractall(backup_folder_path)
assert os.path.isdir(backup_folder_path)

# Make dataset folders
save_path_csv = "../dataset/data_final"
os.mkdir(save_path_csv)
save_path = "../dataset/data_final/data"
os.mkdir(save_path)

# Extract project labels
project_settings_path = glob(os.path.join(backup_folder_path, "*.json"))
assert len(project_settings_path) == 1

df_project_settings = pd.read_json(project_settings_path[0])
project_labels = df_project_settings["labels"].apply(lambda x: x["name"]).to_list()
project_labels_map = {label:i for i, label in enumerate(project_labels)}
assert len(project_labels) == 15

# Tasks paths
task_folder_paths = [task_path for task_path in glob(os.path.join(backup_folder_path, "*")) if task_path[-5:] != ".json"]
task_folder_paths = sorted(task_folder_paths, key=lambda x: int(x.split(os.sep)[-1].split("_")[-1])) # Number sort

# Setup - dataframes

In [ ]:
# Load video info (this is used to add time info)
df_video_info = pd.read_csv("../video_data/video_info.csv")
df_video_info["mapping_key"] = df_video_info["mapping_key"].apply(lambda x: x.replace("/", "_"))
df_video_info["mapping_key"] = df_video_info["mapping_key"].apply(lambda x: x.replace(".MP4", ""))
df_video_info["video_file_name"] = df_video_info["video_file_name"].apply(lambda x: x.replace(".MP4", ""))

# This is just a hack to get a empty dataframe with the same columns
df_annotations = df_video_info.loc[0:-1, :"date_minut"]
df_annotations["annotation_name"] = ""
df_annotations["frame_name"] = ""
df_annotations["uncropped"] = False

for col_name in project_labels:
    df_annotations[col_name] = 0

# Helpers

In [ ]:
def extract_frames_from_video(video_path:str, frames:list):
    
    # Check
    assert os.path.exists(video_path) and (video_path[-4:].lower() == ".mp4"), "Bad video path"
    
    # Setup
    cap = cv2.VideoCapture(video_path)
    frame_i = -1
    return_frames = []
    
    # Extract and save individual frames as specified by the list `frames`
    while cap.isOpened():
        frame_i += 1
        video_feed_active, frame = cap.read()
        
        if not video_feed_active:
            cap.release()
            cv2.destroyAllWindows()
            break
            
        if frame_i in frames:
            return_frames.append(frame)
    
    return return_frames
    
def xyxy2xywhn(bb:list, label:int, img_width:int, img_height:int):
    # Setup
    x1, y1, x2, y2 = bb
    bb_width, bb_height = (x2 - x1), (y2 - y1)

    # Width and height
    bb_width_norm = bb_width / img_width
    bb_height_norm = bb_height / img_height

    # Center
    bb_center_x_norm = (x1 + bb_width / 2) / img_width
    bb_center_y_norm = (y1 + bb_height / 2) / img_height

    # Yolo format --> `class_name_int center_x center_y width height`
    string = str(label)
    for s in [bb_center_x_norm, bb_center_y_norm, bb_width_norm, bb_height_norm]:
        string += " " + str(s)

    return string            

def extract_bb_info(annotations:list, video_width, video_height):
    to_return = {}
    bbs_xywhn = []
    labels = []
    
    for annotation in annotations:
        # Frame number
        frame_number = annotation["frame"]
        
        # Label
        label_int = project_labels_map[annotation["label"]]
        labels.append(label_int)
        
        # Bounding boxes
        bb_xywhn = xyxy2xywhn(annotation["points"], label_int, video_width, video_height)
        
        if frame_number in to_return.keys():
            to_return[frame_number] += "\n" + bb_xywhn
        else:
            to_return[frame_number] = bb_xywhn
    
    return to_return

# Create dataset with csv-file

In [ ]:
for task_folder_path in tqdm(task_folder_paths):
    is_full_size = False
    
    # Load video path
    video_path = list(set(glob(os.path.join(task_folder_path, "data/*.MP4")) + glob(os.path.join(task_folder_path, "data/*.mp4"))))
    assert len(video_path) == 1, "Expected to find exatcly 1 video"
    video_path = video_path[0]
    video_info = U.videos.get_video_info(video_path)
    
    # Extract video name (need to know the location and date)
    with open(os.path.join(task_folder_path, "task.json"))as f:
        task_name = json.load(f)["name"]
        
        # Handle the uncropped videos
        if task_name[-10:] == '_full_size':
            task_name = task_name[:-10]
            is_full_size = True
        
        match = df_video_info[df_video_info["mapping_key"] == task_name]
        assert len(match) == 1, "Expected to find exactly 1 match"
        full_name = match["video_file_name"].values[0]
        pass
    
    # Annotations
    annotation_path = os.path.join(task_folder_path, "annotations.json")
    with open(annotation_path)as f:
        annotations = json.load(f)
        assert len(annotations) == 1
        annotations = annotations[0]["shapes"] # Just the format cvat has chosen
    
    # Frames and corresponding bbs
    frame_2_bbs = extract_bb_info(annotations, video_info["width"], video_info["height"])
    bb_matching_frames = extract_frames_from_video(video_path, list(frame_2_bbs.keys()))
    assert len(frame_2_bbs) == len(bb_matching_frames), "Shape mismatch between frames and bounding boxes"
    
    # Save frames (.png) and annotations (.txt) in yolo-format (class_label x_center, y_center, width, heghti)
    for (image, frame_i, bbs) in zip(bb_matching_frames, list(frame_2_bbs.keys()), list(frame_2_bbs.values())):
        name_general = full_name + "_" + str(frame_i)
        save_path_general = os.path.join(save_path, name_general)
        
        # Write to disk
        with open(save_path_general+".txt", 'x') as f:
            print(bbs, file=f, end="")
        cv2.imwrite(save_path_general+".png", image)
        
        # Update annotation dataframe with relevant info
        to_append = match.loc[:, :"date_minut"]
        to_append["annotation_name"] = name_general + ".txt"
        to_append["frame_name"] = name_general + ".png"
        to_append["uncropped"] = is_full_size
        
        labels = [int(bb.split(" ")[0]) for bb in bbs.split("\n")]
        for (label_name, label_i) in project_labels_map.items():
            to_append[label_name] = labels.count(label_i)
        
        df_annotations = df_annotations.append(to_append)

df_annotations = df_annotations.reset_index(drop=True)
df_annotations.to_csv(os.path.join(save_path_csv, "info.csv"), index=False)

# Create .txt file with label_name:label_int


In [ ]:
string = ""
for k, v in project_labels_map.items():
    string += f"{k}:{v}\n"

with open("../dataset/data_final/labels.txt", "x") as f:
    print(string.strip(), file=f, end="")

# Testing

In [ ]:
images_with_bb = []
for _ in range(10):
    random_name = random.choice(df_annotations["frame_name"].to_list())[:-4]
    image_path = glob(f"{save_path}/{random_name}.png")[0]
    anno_path = glob(f"{save_path}//{random_name}.txt")[0]
    image_drawn = U.pytorch.yolo_draw_bbs_path(image_path, anno_path)
    images_with_bb.append(image_drawn)
show(images_with_bb)